In [2]:
library(ggplot2)
library(reshape2)
library(dplyr)
library(stringr)
library(tidyr)
theme_set(theme_bw())
options(repr.plot.width=7, repr.plot.height=4)


Attaching package: ‘tidyr’

The following object is masked from ‘package:reshape2’:

    smiths



In [8]:
identities = read.delim('identities.tsv', sep='\t', stringsAsFactors=FALSE)
identities$quality = as.logical(identities$quality)
identities$restrict = as.logical(identities$restrict)

# Introduction

Here, I measure base frequencies and tabulate identity elements by clade. Biologists in the past have named things as identity elements if 100% of tRNAs have this element. I'll relax those restrictions to 98% among the quality and species-restricted tRNAs.

Terminology:
- An **identity element** is a base or base class (G, purine, etc.) located at a specific position.
- **Consensus** identity elements for a clade means that all subclades contain this identity element.
- **Diverged** positions for a clade means that all subclades contain an IDE at that position, even if the IDE differs. Of course, the IDE itself could be "zoomed out", to a lower specificity, and then the position wouldn't be described as diverged anymore. For example, if archaea and bacteria have A41 but eukarya have G41, then position 41 is diverged if you consider A41 and G41 to be separate IDEs, but not diverged if you call it as R41.
- **Unique** identity elements for a subclade means that all sister subclades do not have an IDE at this position.
- We could potentially have a case where certain subclades have an IDE at a position while others do not. These will be known as **shared** identity elements among two or more unrelated subclades.

## Base combination table

Reproduced from euk-Leu

Symbol   |   Bases  | Rule  | Rank
:--------|----------|-------|------
R | G A | Purine | 2
Y | U C | Pyrimidine | 2
K | G U | Keto | 2
M | A C | Amino | 2
S | G C | Strong | 2
W | A U | Weak | 2
B | C G U | not A | 3
D | G A U | not C | 3
H | A C U | not G | 3
V | G C A | not U | 3
R:Y | G:C A:U | Purine-Pyrimidine | 2
Y:R | C:G U:A | Pyrimidine-Purine | 2
S:S | G:C C:G | Strong | 2
W:W | A:U U:A | Weak | 2
W:O | G:U U:G | Wobble | 2
B:V | G:C C:G U:A | not A:U | 3
V:B | G:C C:G A:U | not U:A | 3
D:H | A:U U:A G:C | not C:G | 3
H:D | A:U U:A C:G | not G:C | 3
W:C | A:U U:A G:C C:G | Watson-Crick | 4
G:N | G:C G:U C:G U:G | G | 5
U:N | U:G U:A A:U G:U | U | 5
C:O | A:U U:A G:C C:G G:U U:G | Canonical pairs | 6
Y:Y | U:C C:U U:U C:C | Pyrimidine-pyrimidine mismatch | 7
R:R | A:G G:A A:A G:G | Purine-purine mismatch | 7
M:M | A:A G:G C:C U:U A:G A:C C:A C:U G:A U:C | Mismatch | 8

# Consensus eukaryotic identity elements

I will start with the easiest case: IDEs that are almost perfectly conserved among all eukaryotes.

## ACGT

In [4]:
positions = colnames(identities)[which(str_detect(colnames(identities), 'X\\d+\\.'))]
positions = c(positions, 'X8', 'X9', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X20a', 'X21', 'X26', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X44', 'X45', 'X46', 'X47', 'X48', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X73')

In [48]:
euk_freqs = identities %>% select(match(positions, colnames(identities))) %>%
  gather(positions, bases) %>%
  group_by(positions, bases) %>%
  tally() %>%
  group_by(positions) %>%
  mutate(freq=n/sum(n))

In [49]:
euk_ides = euk_freqs %>% filter(freq > 0.95)
euk_freqs = euk_freqs %>% filter(!(positions %in% euk_ides$positions))
euk_ides

,positions,bases,n,freq
1,X14,A,109604,0.9942488
2,X18,G,109042,0.9891507
3,X18.55,G:U,107878,0.9785918
4,X19,G,108902,0.9878808
5,X21,A,108344,0.982819
6,X33,U,107716,0.9771222
7,X54,U,107175,0.9722147
8,X55,U,109063,0.9893412
9,X58,A,107426,0.9744916
10,X8,U,107368,0.9739654


## Purine, pyrimidine, keto, amino, strong, and weak

In [52]:
euk_freqs

,positions,bases,n,freq
1,X10.25,-:A,1,9.071282e-06
2,X10.25,A:A,36,0.0003265662
3,X10.25,A:C,284,0.002576244
4,X10.25,A:U,516,0.004680782
5,X10.25,C:A,9,8.164154e-05
6,X10.25,C:C,134,0.001215552
7,X10.25,C:G,306,0.002775812
8,X10.25,C:U,44,0.0003991364
9,X10.25,-:G,1,9.071282e-06
10,X10.25,G:A,762,0.006912317


In [62]:
euk_freqs %>% filter(positions %in% c('X9', 'X20')) %>%
  group_by(positions) %>%
  summarize(purine = sum(freq[bases %in% c("A", "G")]), pyrimidine = sum(freq[bases %in% c("A", "G")])

ERROR: Error in parse(text = x, srcfile = src): <text>:4:0: unexpected end of input
2:   group_by(positions) %>%
3:   summarize(purine = sum(freq[bases %in% c("A", "G")]), pyrimidine = sum(freq[bases %in% c("A", "G")])
  ^
